# Дообучение Gemma-2-2b с помощью Keras

Сохраним в переменных окружения KAGGLE_USERNAME и KAGGLE_KEY из .env файла.

In [7]:
import kaggle

kaggle.api.authenticate()

Установка зависимостей

In [8]:
# Install Keras 3 last. See https://keras.io/getting_started/ for more details.
# %pip install -q keras-nlp
# %pip install -q keras
# %pip install -q jax

Используем JAX как бэкэнд для Keras.

In [9]:
import os

os.environ["KERAS_BACKEND"] = "jax"  # Or "torch" or "tensorflow".
# Avoid memory fragmentation on JAX backend.
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "1.00"

Импортируем Keras.

In [10]:
import keras
import keras_hub

TypeError: Unable to convert function return value to a Python type! The signature was
	() -> handle

Скачаем и сохраним датасет.

In [ ]:
!wget -q -O databricks-dolly-15k.jsonl https://huggingface.co/datasets/databricks/databricks-dolly-15k/resolve/main/databricks-dolly-15k.jsonl

Сделает список строк, типа "Instruction: ... Response: ...", разделённых символами новой строки "\n".

In [ ]:
import json

data = []
with open("databricks-dolly-15k.jsonl") as file:
    for line in file:
        features = json.loads(line)
        # Filter out examples with context, to keep it simple.
        if features["context"]:
            continue
        # Format the entire example as a single string.
        template = "Instruction:\n{instruction}\n\nResponse:\n{response}"
        data.append(template.format(**features))

# Only use 1000 training examples, to keep it fast.
data = data[:1000]
data[0]

'Instruction:\nWhich is a species of fish? Tope or Rope\n\nResponse:\nTope'

In [ ]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma2_2b_en")
gemma_lm.summary()

AttributeError: module 'keras.api.models' has no attribute 'GemmaCausalLM'

In [ ]:
prompt = template.format(
    instruction="What should I do on a trip to Europe?",
    response="",
)
sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)
gemma_lm.compile(sampler=sampler)
print(gemma_lm.generate(prompt, max_length=256))